In [ ]:
import requests 
import bs4 as bs 
import pandas as pd 
import numpy as np

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

## Yahoo News

In [ ]:
link = []
page_content = []

source = requests.get('https://news.search.yahoo.com/search?p=ai+music+composer&fr=uh3_news_vert_gs&fr2=p%3Anews%2Cm%3Asa&guce_referrer=aHR0cHM6Ly9uZXdzLnlhaG9vLmNvbS8&guce_referrer_sig=AQAAAJ8sWO8YODsfi--VCmmI1a3VYjfuKUHL_sGUZGFEBaRDVCGo0azFsVYNtJZSvgj0KLj5E-SypMw7Rv3mB8hNv3SZAiO4XI-uwJPuFk98Ezgil9aQVjx3zs0zkdtgmV5krQNlDFuKfoz8uP8YBLsnDv6GnwPj5BGZPfezu3Qkfcfd&_guc_consent_skip=1584070602')
source.encoding = 'utf-8'
soup = bs.BeautifulSoup(source.content , features = 'html.parser')
news_container = soup.find('div',{'id':'web'})

try:
    for news in news_container.find_all('div',{'class':'dd NewsArticle'}):# find all (list)
        # get links 
        url_content = news.find('a',{"class":"thmb"}).get('href')
        link.append(url_content)
        
        # get text of each page 
        new_source = requests.get(url_content)
        new_source.encoding = 'utf-8'
        new_soup = bs.BeautifulSoup(new_source.content , features = 'html.parser')
        
        ps = new_soup.find_all('p')
       
        page_texts = []
        for p in ps:
            page_texts.append(p.text)
        page_content.append(' '.join(page_texts))
              
except:
    print('None')

In [ ]:
import textblob
sentiment = []
for content in page_content:
    blob = textblob.TextBlob(content)
    blob.sentences
    sentiment.append(blob.sentiment)
    print(blob.sentiment)

In [ ]:
YH_URL_dict = {'URL':link,
              'Content':page_content,
            'Sentiment':sentiment}


YH_URL_frame = pd.DataFrame(YH_URL_dict)
YH_URL_frame.to_csv('Yahoo sentiment.csv',encoding = 'utf-8')

In [ ]:
YH_URL_frame

----------
## Google News

In [ ]:
main_page = requests.get('https://news.google.com/search?q=AI%20composer%20music&hl=en-US&gl=US&ceid=US%3Aen')
main_page.encoding = 'utf-8'

main_soup = bs.BeautifulSoup ( main_page.content , features='html.parser')
print(main_soup.prettify)

In [ ]:
# create empty lists to receive values
title = []
sources = []
link = []

source = requests.get('https://news.google.com/search?q=AI%20composer%20music&hl=en-US&gl=US&ceid=US%3Aen')
source.encoding = 'utf-8'
soup = bs.BeautifulSoup(source.content , features = 'html.parser')
news_container = soup.find('div',{'class':'lBwEZb BL5WZb xP6mwf'})

try:
    for news in news_container.find_all('h3',{"class":"ipQwMb ekueJc RD0gLb"}):# find all (list)
        title_content = news.text
        title.append(title_content)
except:
    title.append('None')

try:
    for news in news_container.find_all('a',{"class":"wEwyrc AVN2gc uQIVzc Sksgp"}):
        sources_content = news.text
        sources.append(sources_content)
except:
    sources.append('None')

try:
    for news in news_container.find_all('div',{"class":"xrnccd"}):
        link_content = news.find('a').get('href')
        link.append(link_content.replace('.','https://news.google.com'))
except:
    link.append('None')
    

In [ ]:
URL_dict = {'Title':title,
            'Source':sources,
            'URL':link}

URL_frame = pd.DataFrame(URL_dict)
URL_frame.to_csv('Google URLs.csv',encoding = 'utf-8')

In [ ]:
URL_frame

In [ ]:
URL_frame['URL']

# get text of each page 
        
page_content2 = []
for URL in URL_frame['URL']:
    print(URL)
    
    new_source = requests.get(URL)
    new_source.encoding = 'utf-8'
    new_soup = bs.BeautifulSoup(new_source.content , features = 'html.parser')
        
    ps = new_soup.find_all('p')
       
    page_texts = []
    for p in ps:
        page_texts.append(p.text)
    page_content2.append(' '.join(page_texts))
    

In [ ]:
import textblob
sentiment2 = []
for content in page_content2:
    blob = textblob.TextBlob(content)
    blob.sentences
    sentiment2.append(blob.sentiment)
    print(blob.sentiment)

In [ ]:
GO_URL_dict = {'URL':URL_frame['URL'],
              'Content':page_content2,
            'Sentiment':sentiment2}


GO_URL_frame = pd.DataFrame(GO_URL_dict)
GO_URL_frame.to_csv('Google sentiment.csv',encoding = 'utf-8')

In [ ]:
GO_URL_frame

In [ ]:
URL_sentiment_frame = pd.concat([YH_URL_frame,GO_URL_frame])

In [ ]:
URL_sentiment_frame = URL_sentiment_frame.reset_index(drop=True)
URL_sentiment_frame.to_csv('Total Setiment.csv',encoding = 'utf-8')